### 7. Предельно допустимая рекреационная емкость особо охраняемой природной территории (RСCоопт) рассчитывается с помощью функции:

In [1]:
import math
import math
import osmnx as ox
import folium
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'osmnx'

*Задайте область интереса. В вашем случае это Камчатка и конкретно объекты, относящиеся к особо охраняемым территориям. Вы можете загрузить данные с определенными тегами:*

In [ ]:
# Установите URL для Overpass API (если необходимо)
ox.settings.overpass_endpoint = "http://overpass-api.de/api/"

# Укажите название области
place_name = "Камчатский край, Россия"

# Получите границы особо охраняемых природных территорий
oopt = ox.geocode_to_gdf(place_name)
map_center = [oopt.geometry.centroid.y, oopt.geometry.centroid.x]
oopt_polygons = ox.geometries_from_place(place_name, tags={'boundary': 'protected_area', 'protect_class': '*'})

# Загружаем точки, помеченные как памятники природы
points = ox.geometries_from_place(place_name, tags={'leisure': 'nature_reserve', 'tourism':'nature_reserve', 'natural':['peak','volcano'],'historic':'monument'})
points = points[points.geometry.type == 'Point']

map_center = [56.0, 160.0] # Центр Камчатки по умолчанию

# Создайте карту с фокусом на загруженную область
m = folium.Map(location=map_center, zoom_start=6)

# Слои с возможностью отключения
feature_group_oopt = folium.FeatureGroup(name="Особо охраняемые природные территории")
feature_group_points = folium.FeatureGroup(name="Памятники природы")

# Добавьте границы особо охраняемых территорий на карту
for index, row in oopt_polygons.iterrows():
    popup_content = f"<b>Название:</b> {row.get('name', 'Без названия')}<br>" \
                    f"<b>Класс защиты:</b> {row.get('protect_class', 'Не указан')}<br>" \
                    f"<b>Оффициальное название:</b> {row.get('official_name', 'Не указан')}<br>" \
                    f"<b>Дата основания:</b> {row.get('start_date', 'Не указан')}<br>" \
                    f"<b>Веб-сайт:</b> {row.get('website', 'Не указан')}<br>"

    folium.GeoJson(
        mapping(row['geometry']),
        popup=folium.Popup(popup_content, max_width=300),
        tooltip=row.get('name', 'ООПТ'),
        # style_function=lambda x: {'fillColor': '#00FF00', 'color': '#008000', 'fillOpacity': 0.5}
    ).add_to(feature_group_oopt)


# Добавляем маркеры памятников природы
for index, row in points.iterrows():
    folium.Marker(
        location=[row['geometry'].y, row['geometry'].x],
        popup=row.get('name', 'Без названия'),
        tooltip=row.get('name', 'Памятник природы')
    ).add_to(feature_group_points)


feature_group_oopt.add_to(m)
feature_group_points.add_to(m)

folium.LayerControl().add_to(m) # Кнопка управления слоями

# Отобразите карту
m

In [ ]:
def calculation_RCCoopt(features: list[int], RCCqm: list[int]) -> tuple[dict, int]:
    '''
    RCCqm - предельно допустимая рекреационная емкость туристского объекта m, человек в единицу времени;
    features - список туристских объектов (0...m-1).
    '''

    # Создаем словарь с ключами из списка номеров (или названий) объектов и рассчитанными значениями RCCq
    dict_features_RCCqm = {feature: rccq for feature, rccq in zip(features, RCCqm)}
    
    result = int(sum(dict_features_RCCqm.values()))
    return dict_features_RCCqm, result

### 8. Предельно допустимая рекреационная емкость туристского объекта (RCCq) рассчитывается с помощью функции:

In [ ]:
def calculation_RCCq(PCCq: list[int], MC: list[float]) -> list[int]:
    '''
    PCCq - потенциальная рекреационная емкость туристского объекта, человек в единицу времени;
    MC - коэффициент управленческой емкости, долей от единицы.
    '''
    
    result = []
    
    for pccq, mc in zip(PCCq, MC):
        result.append(int(pccq * mc))
    
    return result

### 9. Потенциальная рекреационная емкость рассчитывается для особо охраняемой природной территории в целом, а также для ее отдельных частей (туристских объектов) и выражается в целочисленных значениях, человек в единицу времени.

### 10. Потенциальная рекреационная емкость туристского объекта (PCCq) рассчитывается c помощью функции:

In [ ]:
def calculation_PCCq(BCCq: list[int], Cfn: list[list[int]]) -> list[int]:
    '''
    BCCq - базовая рекреационная емкость туристского объекта, выраженная в целочисленном значении, человек в единицу времени;
    Cfn - поправочные коэффициенты, которые учитывают определенные для туристских объектов лимитирующие факторы развития туризма (экологического, социального и социокультурного характера) и установленные режимы использования туристских объектов;
    n - количество поправочных коэффициентов. 
    '''

    # Перемножаем элементы вложенных в список Cf списков
    Cf_multiplied_sublists = [math.prod(sublist) for sublist in Cfn]
    
    result = []
    
    for bccq, cf in zip(BCCq, Cf_multiplied_sublists):
        result.append(int(bccq * cf))

    return result

### 11. Базовая рекреационная емкость туристских объектов (BCC q) для площадных и линейных туристских объектов (туристских маршрутов) рассчитывается по приведенным формулам и выражается в целочисленных значениях, человек в единицу времени.

### 12. Базовая рекреационная емкость для ПЛОЩАДНЫХ туристских объектов (BCC qs) рассчитываетсяс помощь функции:

In [ ]:
def calculation_BBCqs(A: list[float], Au: list[float], Rf: list[float], t: list[int]) -> list[int]:
    '''
    A - площадь туристского объекта, на которой осуществляется туризм, кв. метров;
    Au - площадь туристского объекта, необходимая для одного посетителя при осуществлении туризма (кв. метров), определяемая в соответствии с порядком, указанным в пункте 18 настоящих Правил;
    Rf - коэффициент возвращения, отражающий возможное количество посещений туристского объекта одним и тем же туристом в день;
    t - количество дней в рассматриваемую единицу времени (месяц, сезон, год и др.), единиц.
    '''
    
    result = []
    
    for area, area_per_visitor, return_factor, units_of_time in zip(A, Au, Rf, t):
        bbcq = (area / area_per_visitor) * return_factor * units_of_time 
        result.append(int(bbcq)) 

    return result


### 13. Коэффициент возвращения (Rf) рассчитывается по формуле:

In [ ]:
def calculation_Rf(T: list[int], Td: list[int]) -> list[float]:
    '''
    Т - количество часов в сутки, когда туристский объект доступен для посещения, часов;
    Td - среднее время пребывания посетителя на туристском объекте, часов.
    '''

    result = []
    
    for times, mean_times in zip(T, Td):
        rf = times / mean_times 
        result.append(float(rf)) 

    return result

### 14. Базовая рекреационная емкость для ЛИНЕЙНЫХ: однодневных туристских маршрутов и многодневных туристских маршрутов с небольшой протяженностью или несколькими входами без ограничения времени посещения туристского маршрута (BCC qpi) рассчитывается по формуле:

In [ ]:
def calculation_BBCqpi(
        DTp: list[list[float]], 
        DGp: list[list[float]], 
        Ts: list[list[float]],
        Tdp: list[list[float]],
        GS: list[list[float]],
        tp: list[int]
        ) -> list[float]:
    '''
    DTp - протяженность однодневного туристского маршрута или однодневного участка p многодневного туристского маршрута в дневной переход, км;
    DGp - оптимальное расстояние между группами на участке p туристского маршрута, км;
    Ts - длина светового дня или количество времени, когда туристский маршрут доступен для посетителей, часов;
    Tdp - среднее время прохождения участка туристского маршрута p с учетом остановок, часов;
    GS - среднее количество человек в группе (включая сопровождающих), человек;
    p - порядковый номер однодневного участка туристского маршрута (1, 2, ... p);
    tp - количество дней пребывания посетителей на туристском маршруте, единиц.
    '''
    if Tdp <= 0:
        raise ValueError("Среднее время прохождения участка (Td_p) должно быть больше 0.")
    
    # Расчет количества групп, которое может пройти за световой день
    num_groups_per_day = Ts / Tdp
    
    # Общая рекреационная емкость
    capacity = (DTp / DGp) * num_groups_per_day * GS * tp
    
    return capacity

# Пример использования функции
DTp = 10.0  # длина маршрута в км
DGp = 1.0   # оптимальное расстояние между группами в км
Ts = 12.0    # длина светового дня в часах
Tdp = 2.0   # среднее время прохождения участка в часах
GS = 5       # среднее количество человек в группе
tp = 2      # количество дней пребывания

result = calculation_BBCqpi(DTp, DGp, Ts, Tdp, GS, tp)
print(f"Базовая рекреационная емкость: {result}")

Базовая рекреационная емкость: 600.0


In [ ]:
# Определяем список туристических объектов (!!! Все три объекта ПЛОЩАДНЫЕ !!!)
tourist_features = ['Объект 1', 'Объект 2', 'Объект 3']

# Определяем коэффициенты возвращения туристов на объекты
Rf_list = calculation_Rf([8, 7, 24], [5, 7, 12])
print(f'Коэффициенты возвращения туристов на объекты: {Rf_list}, единиц времени')

# Определяем базовую рекреационную емкость для туристских объектов ЕСЛИ ВСЕ ОНИ ПЛОЩАДНЫЕ 
# !!! пока логика разделения на линейные и площадные объекты на реализована и 
# все объекты должна задаваться одним типом: линейным или площадным !!!
BBCqs_list = calculation_BBCqs([
                                100.2, 50.0, 25.1
                                ], 
                                [
                                30.5, 12.0, 25.0
                                ], 
                                Rf_list, 
                                [
                                31, 30, 28
                                ])
print(f'Базовые рекреационные емкости: {BBCqs_list}, человек в единицу времени')

# Определяем потенциальную рекреационную емкость для каждого объекта
PCCq_list = calculation_PCCq(BBCqs_list, [[0.4, 0.5], [0.5, 2, 0.3], [0.8]])   
print(f'Потенциальные рекркационные емкости: {PCCq_list}, человек в единицу времени')

# Определяем коэффициенты управленческой емкости для каждого объекта
MC_list = [0.8, 0.5, 0.75]

# Вычисляем список придельно допустимых туристских емкостей объектов
RCCq_list = calculation_RCCq(PCCq_list, MC_list)
print(f'Придельно допустимые туристские емкости: {RCCq_list}, человек в единицу времени')

# Пусть у нас m туристических объекта c соответствующими предельно допустимыми рекреационными емкостями, 
# тогда предельно допустимая рекреационная емкость особо охраняемой природной территории RCCоопт будет равна   
RCCoopt_int = calculation_RCCoopt(tourist_features, RCCq_list)
print(f'Предельно допустимая рекреационная емкость ООПТ: {RCCoopt_int}, человек в единицу времени')


Коэффициенты возвращения туристов на объекты: [1.6, 1.0, 2.0], единиц времени
Базовые рекреационные емкости: [162, 125, 56], человек в единицу времени
Потенциальные рекркационные емкости: [32, 37, 44], человек в единицу времени
Придельно допустимые туристские емкости: [25, 18, 33], человек в единицу времени
Предельно допустимая рекреационная емкость ООПТ: ({'Объект 1': 25, 'Объект 2': 18, 'Объект 3': 33}, 76), человек в единицу времени


In [ ]:
# Определяем список туристических объектов (!!! Все три объекта ЛИНЕЙНЫЕ !!!)
tourist_features = ['Объект 1', 'Объект 2', 'Объект 3']

# Определяем коэффициенты возвращения туристов на объекты
Rf_list = calculation_Rf([8, 7, 24], [5, 7, 12])
print(f'Коэффициенты возвращения туристов на объекты: {Rf_list}, единиц времени')

# Определяем базовую рекреационную емкость для туристских объектов ЕСЛИ ВСЕ ОНИ ПЛОЩАДНЫЕ 
# !!! пока логика разделения на линейные и площадные объекты на реализована и 
# все объекты должна задаваться одним типом: линейным или площадным !!!
BBCqs_list = calculation_BBCqs([
                                100.2, 50.0, 25.1
                                ], 
                                [
                                30.5, 12.0, 25.0
                                ], 
                                Rf_list, 
                                [
                                31, 30, 28
                                ])
print(f'Базовые рекреационные емкости: {BBCqs_list}, человек в единицу времени')

# Определяем потенциальную рекреационную емкость для каждого объекта
PCCq_list = calculation_PCCq(BBCqs_list, [[0.4, 0.5], [0.5, 2, 0.3], [0.8]])   
print(f'Потенциальные рекркационные емкости: {PCCq_list}, человек в единицу времени')

# Определяем коэффициенты управленческой емкости для каждого объекта
MC_list = [0.8, 0.5, 0.75]

# Вычисляем список придельно допустимых туристских емкостей объектов
RCCq_list = calculation_RCCq(PCCq_list, MC_list)
print(f'Придельно допустимые туристские емкости: {RCCq_list}, человек в единицу времени')

# Пусть у нас m туристических объекта c соответствующими предельно допустимыми рекреационными емкостями, 
# тогда предельно допустимая рекреационная емкость особо охраняемой природной территории RCCоопт будет равна   
RCCoopt_int = calculation_RCCoopt(tourist_features, RCCq_list)
print(f'Предельно допустимая рекреационная емкость ООПТ: {RCCoopt_int}, человек в единицу времени')